<a href="https://colab.research.google.com/github/MihaelaCatan04/Sigmoid_Deep_Dive/blob/main/Lesson_3/Lesson_3_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Homework Nr. 3
#### Tasks:
1. Complete week 3 of second course and week 1 of third course from specialization;
2. As midterms approach and I know some of you have to finish past weeks and past homeworks, I will make the practical task optional.

#### Practical Task:
Create a Spam detection system using Pytorch. Input: text (SMS, email, etc.), Output: spam or not spam, 2 classes.

Hint: You can use the [SMS Spam Collection Dataset](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset) from Kaggle.
Hint x2: As Neural Networks work with numbers, you will need to convert the text to numbers first. You can use various vectorization techniques you can find on the internet, or watch some past youtube videos from Sigmoid NLPC bootcamp!

Good luck!

### Solution


In [1]:
# Connect to Google Drive
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
!mkdir -p ~/.kaggle
!cp drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
# Download the dataset
!kaggle datasets download -d uciml/sms-spam-collection-dataset

Dataset URL: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
License(s): unknown
100% 211k/211k [00:00<00:00, 400kB/s]
100% 211k/211k [00:00<00:00, 400kB/s]


In [3]:
# Unzip dataset
!unzip sms-spam-collection-dataset.zip

Archive:  sms-spam-collection-dataset.zip
  inflating: spam.csv                


In [4]:
# Install WAndB
!pip install wandb

In [5]:
# Login on WAndB
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
import wandb

In [7]:
import chardet

# Read a small sample of the file to detect the encoding
with open("spam.csv", "rb") as file:
    result = chardet.detect(file.read(10000))
print(result['encoding'])

# Use the detected encoding to read the file
data = pd.read_csv("spam.csv", encoding=result['encoding'])

# Display data
data.head()

Windows-1252


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
# Drop the useless columns
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],
  axis='columns', inplace=True)

In [9]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
# Encode the column to be predicted
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(data['v1'].unique())
le.classes_
data['v1'] = le.fit_transform(data['v1'])

In [11]:
data.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
# Import tokenizer
import nltk
from nltk.tokenize import word_tokenize

In [13]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [14]:
# Apply tokenizer to the rest of the data
data['v2'] = data['v2'].apply(word_tokenize)

In [15]:
data.head()

,v1,v2
0,0,"[Go, until, jurong, point, ,, crazy, .., Avail..."
1,0,"[Ok, lar, ..., Joking, wif, u, oni, ...]"
2,1,"[Free, entry, in, 2, a, wkly, comp, to, win, F..."
3,0,"[U, dun, say, so, early, hor, ..., U, c, alrea..."
4,0,"[Nah, I, do, n't, think, he, goes, to, usf, ,,..."


In [16]:
# Initialize the stemmer
stemmer = PorterStemmer()

# Function to apply stemming to the data
def apply_stemming(text):
    stemmed_words = [stemmer.stem(word) for word in text]  # Stem each word
    return ' '.join(stemmed_words)

In [17]:
# Apply stemming to the 'v2' column
data['v2'] = data['v2'].apply(apply_stemming)

# Display the DataFrame with the new column
print(data.head())

   v1                                                 v2
0   0  go until jurong point , crazi .. avail onli in...
1   0                      ok lar ... joke wif u oni ...
2   1  free entri in 2 a wkli comp to win fa cup fina...
3   0  u dun say so earli hor ... u c alreadi then sa...
4   0  nah i do n't think he goe to usf , he live aro...


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the 'v2' column and transform it
tfidf_matrix = vectorizer.fit_transform(data['v2'])

# Convert the TF-IDF matrix to a dense DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=vectorizer.get_feature_names_out()
)

# Replace the 'v2' column with the TF-IDF features
data = pd.concat([data.drop(columns=['v2']), tfidf_df], axis=1)

data.head()

,v1,00,000,000pe,008704050406,0089,0121,01223585236,01223585334,0125698789,...,û_,û_thank,ûªm,ûªt,ûªv,ûï,ûïharri,ûò,ûówel,žö
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Split dataset
X = data.drop("v1", axis=1).values
y = data["v1"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Dataset class
class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [21]:
# DataLoaders
train_dataset = SpamDataset(X_train, y_train)
test_dataset = SpamDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [22]:
# Shape of dataloaders
for X, y in test_loader:
  print(f"Shape of X: {X.shape}")
  print(f"Shape of y: {y.shape}")
  break

Shape of X: torch.Size([32, 7423])
Shape of y: torch.Size([32])


In [23]:
# Use CUDA if available
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.mps.is_available()
    else "cpu"
)

print(f"Your laptop is using {device}")

Your laptop is using cuda


In [24]:
# Neural Network made of ReLU and Sigmoid Functions
class NeuralNetworkReLUSigmoid(nn.Module):
    def __init__(self):
        super(NeuralNetworkReLUSigmoid, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_sigmoid_stack = nn.Sequential(
            nn.Linear(7423, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.flatten(x)
        predictions = self.linear_relu_sigmoid_stack(x)
        return predictions


In [25]:
# Train function to train the neural network
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    total_loss = 0

    for batch_no, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.unsqueeze(1).to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_no % 5 == 0 or batch_no == num_batches - 1:
            current = min((batch_no + 1) * len(X), size)
            print(f"Batch {batch_no + 1}/{num_batches} - Loss: {loss.item():.4f} [{current}/{size}]")

    avg_loss = total_loss / num_batches
    print(f"Epoch Training Loss: {avg_loss:.4f}")

In [26]:
# Test function to test how well is our neural network trained
def test(dataloader, model, loss_fn, epoch=None, log_to_wandb=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.unsqueeze(1).to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += ((pred > 0.5) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = correct / size

    if log_to_wandb and epoch is not None:
        wandb.log({
            "epoch": epoch + 1,
            "test_loss": test_loss,
            "test_accuracy": accuracy
        })

    print(f"Test Results:\n Accuracy: {(100 * accuracy):>0.1f}%, Avg Loss: {test_loss:.4f}\n")

In [27]:
# Set a run at WAndB
wandb.init(
    project="neural-networks-homework3",
    config={
        "dataset": "spam-dataset",
        "epochs": 5,
        "batch_size": 32,
        "learning_rate": 0.001,
        "optimizer": "Adam"
    },
    settings=wandb.Settings(init_timeout=120)
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mihaelacatan (mihaela-catan). Use `wandb login --relogin` to force relogin


In [28]:
# Initialize model and move to device
model = NeuralNetworkReLUSigmoid().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.learning_rate)
loss_func = nn.BCELoss()

# Training and Testing Loop
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch+1}\n------------------------------")
    train(train_loader, model, loss_func, optimizer)
    test(test_loader, model, loss_func, epoch, log_to_wandb=True)

# Save model
torch.save(model.state_dict(), "NeuralNetworkReLUSigmoid.pth")
wandb.save("NeuralNetworkReLUSigmoid.pth")

Epoch 1
------------------------------
Batch 1/140 - Loss: 0.6838 [32/4457]
Batch 6/140 - Loss: 0.4898 [192/4457]
Batch 11/140 - Loss: 0.2429 [352/4457]
Batch 16/140 - Loss: 0.7918 [512/4457]
Batch 21/140 - Loss: 0.2453 [672/4457]
Batch 26/140 - Loss: 0.3278 [832/4457]
Batch 31/140 - Loss: 0.3095 [992/4457]
Batch 36/140 - Loss: 0.2884 [1152/4457]
Batch 41/140 - Loss: 0.1033 [1312/4457]
Batch 46/140 - Loss: 0.2505 [1472/4457]
Batch 51/140 - Loss: 0.1824 [1632/4457]
Batch 56/140 - Loss: 0.1849 [1792/4457]
Batch 61/140 - Loss: 0.1321 [1952/4457]
Batch 66/140 - Loss: 0.2367 [2112/4457]
Batch 71/140 - Loss: 0.0848 [2272/4457]
Batch 76/140 - Loss: 0.1870 [2432/4457]
Batch 81/140 - Loss: 0.0713 [2592/4457]
Batch 86/140 - Loss: 0.0541 [2752/4457]
Batch 91/140 - Loss: 0.0789 [2912/4457]
Batch 96/140 - Loss: 0.0506 [3072/4457]
Batch 101/140 - Loss: 0.0095 [3232/4457]
Batch 106/140 - Loss: 0.0960 [3392/4457]
Batch 111/140 - Loss: 0.0598 [3552/4457]
Batch 116/140 - Loss: 0.1982 [3712/4457]
Batch 1

['/content/wandb/run-20241207_204938-e40sgduy/files/NeuralNetworkReLUSigmoid.pth']

**Conclusion**

The task of creating a spam detection system using PyTorch demonstrated the process of text data preparation, feature vectorization, and training a neural network model for binary classification. By using the SMS Spam Collection Dataset, a high accuracy of 97.7% was achieved. The model's performance can be further improved by experimenting with different neural network architectures, incorporating advanced techniques like word embeddings, and using regularization methods to enhance robustness.